### Text classification with torchtext
* https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html
* ngram based

In [33]:
import torch
import torchtext
import torch.nn as nn
import torch.nn.functional as F
from torchtext.datasets import text_classification
from torch.utils.data import DataLoader

import re
from torchtext.data.utils import ngrams_iterator
from torchtext.data.utils import get_tokenizer

from torch.utils.data.dataset import random_split

import time
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Get the data

In [7]:
BATCH_SIZE = 16
NGRAMS = 2

if not os.path.isdir('/ndata/NLP/text_classification'):
    os.mkdir('/ndata/NLP/text_classification')
    
train_dataset, test_dataset = text_classification.DATASETS['AG_NEWS'](
    root='/ndata/NLP/text_classification', ngrams=NGRAMS, vocab=None)

ag_news_csv.tar.gz: 11.8MB [00:00, 65.1MB/s]
120000lines [00:06, 18489.60lines/s]
120000lines [00:14, 8537.20lines/s]
7600lines [00:00, 9216.29lines/s]


### Model definitions

In [17]:
class TextSentiment(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()
        
    def init_weights(self):
        init_range = 0.5
        self.embedding.weight.data.uniform_(-init_range, init_range)
        self.fc.weight.data.uniform_(-init_range, init_range)
        self.fc.bias.data.zero_()
    
    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

### Settings

In [18]:
VOCAB_SIZE = len(train_dataset.get_vocab())
EMBED_DIM = 32
NUN_CLASS = len(train_dataset.get_labels())
model = TextSentiment(VOCAB_SIZE, EMBED_DIM, NUN_CLASS).to(device)

#### Batch functions
* since text entries have different lenghts, we hadd the text lenght ('offset') and pass that to the embedding layer


In [31]:
def generate_batch(batch):
    label = torch.tensor([entry[0] for entry in batch])
    text = [entry[1] for entry in batch]
    offsets = [0] + [len(entry) for entry in text]
    # final offsets via cumsum, note you don't need an offset for the last entry     
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text = torch.cat(text)
    return text, offsets, label


def train_test(dataset_subset, is_train):
    batch_loss = 0
    batch_acc = 0
    data_loader = DataLoader(dataset_subset, batch_size=BATCH_SIZE, shuffle=is_train,
                     collate_fn=generate_batch)
    
    for i, (text, offsets, labels) in enumerate(data_loader):
        if is_train:
            optimizer.zero_grad()
        text, offsets, labels = text.to(device), offsets.to(device), labels.to(device)
        
        if is_train:
            output = model(text, offsets)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
        else:
            with torch.no_grad():
                output = model(text, offsets)
                loss = criterion(output, labels)
        
        batch_loss += loss.item()
        batch_acc += (output.argmax(1) == labels).sum().item()
    
    if is_train:
        scheduler.step()
    
    return batch_loss / (len(dataset_subset)), batch_acc / (len(dataset_subset))


### Main epochs loop
* split dataset
* run epochs

In [32]:
N_EPOCHS = 5
min_valid_loss = float('inf')

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=4.0)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)

train_len = int(len(train_dataset) * 0.95)
sub_train_, sub_valid_ = \
    random_split(train_dataset, [train_len, len(train_dataset) - train_len])

for epoch in range(N_EPOCHS):

    start_time = time.time()
    train_loss, train_acc = train_test(sub_train_, True)
    valid_loss, valid_acc = train_test(sub_valid_, False)

    secs = int(time.time() - start_time)
    mins = secs / 60
    secs = secs % 60

    print('Epoch: %d' %(epoch + 1), " | time in %d minutes, %d seconds" %(mins, secs))
    print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)')
    print(f'\tLoss: {valid_loss:.4f}(valid)\t|\tAcc: {valid_acc * 100:.1f}%(valid)')

Epoch: 1  | time in 0 minutes, 16 seconds
	Loss: 0.0124(train)	|	Acc: 93.4%(train)
	Loss: 0.0148(valid)	|	Acc: 91.6%(valid)
Epoch: 2  | time in 0 minutes, 15 seconds
	Loss: 0.0070(train)	|	Acc: 96.3%(train)
	Loss: 0.0128(valid)	|	Acc: 93.6%(valid)
Epoch: 3  | time in 0 minutes, 15 seconds
	Loss: 0.0039(train)	|	Acc: 98.1%(train)
	Loss: 0.0128(valid)	|	Acc: 93.9%(valid)
Epoch: 4  | time in 0 minutes, 15 seconds
	Loss: 0.0023(train)	|	Acc: 99.0%(train)
	Loss: 0.0142(valid)	|	Acc: 93.7%(valid)
Epoch: 5  | time in 0 minutes, 15 seconds
	Loss: 0.0014(train)	|	Acc: 99.4%(train)
	Loss: 0.0141(valid)	|	Acc: 93.9%(valid)


### Testing on random snippets

In [40]:
ag_news_label = {1 : "World",
                 2 : "Sports",
                 3 : "Business",
                 4 : "Sci/Tec"}


def predict(text, model, vocab, ngrams):
    tokenizer = get_tokenizer("basic_english")
    with torch.no_grad():
        text = torch.tensor([vocab[token]
                            for token in ngrams_iterator(tokenizer(text), ngrams)])
        
        text = text.to(device)
        output = model(text, torch.tensor([0]).to(device))
        return output.argmax(1).item() + 1

In [42]:
vocab = train_dataset.get_vocab()

ex1 = "MEMPHIS, Tenn. – Four days ago, Jon Rahm was \
    enduring the season’s worst weather conditions on Sunday at The \
    Open on his way to a closing 75 at Royal Portrush, which \
    considering the wind and the rain was a respectable showing. \
    Thursday’s first round at the WGC-FedEx St. Jude Invitational \
    was another story. With temperatures in the mid-80s and hardly any \
    wind, the Spaniard was 13 strokes better in a flawless round. \
    Thanks to his best putting performance on the PGA Tour, Rahm \
    finished with an 8-under 62 for a three-stroke lead, which \
    was even more impressive considering he’d never played the \
    front nine at TPC Southwind."

ex2 = "António Guterrez, the United Nations secretary general, \
contrasted the “leadership” and “mobilisation” shown by the world’s \
youth on the climate emergency with the lack of action by governments, \
which were failing to keep up with the urgency of the problem \
despite increasing signs that the climate was reaching breakdown."


# model = model.to("cpu")

print("This is %s news" %ag_news_label[predict(ex1, model, vocab, 2)])
print("This is %s news" %ag_news_label[predict(ex2, model, vocab, 2)])

This is Sports news
This is Sci/Tec news
